# Lab 7
# Ian Grant

In this lab, I used arcpy to automate the process of importing a Geopackage generated by the pipeline in Part 1 into an ESRI Geodatabase. Although ArcGIS Pro can work directly with Geopackages, importing into a Geodatabase allows access to expanded functionality, like topological relationships between the layers. I also extended the pipeline built for Lab 6, as explained below.

## Lab 6 Refactoring

I also repackaged the URL-to-Geopackage pipeline that I made for Lab 6, and I created a script that allows it to be run from the command line in the following format: 

    > python open_data_to_gpkg.py --url [name] [url] --limit [limit] --path [path to geopackage]

Multiple URLs can be specified by adding additional --url tags. The tool also provides status updates to the command line during processing, and it warns the user if the "limit" parameter supplied is too small for the targeted data sources. Please see '/GPKG_pipeline' and 'open_data_to_gpkg.py' for more information. 

## GeoPackage to ESRI Geodatabase processing

In [2]:
import os
import arcpy
import geopandas as gp
import fiona

In [58]:
data_dir = "D:\\bike_data\\"
citibike_gpkg_name = "my_bike_map.gpkg"
citibike_gdb_name = "citibike_data.gdb"

citibike_gpkg_path = os.path.join(data_dir, gpkg_name)

### Listing the layers

The first step is to define a function that returns a list of layers from a Geopackage. The method uses fiona, a package for writing and reading spatial file formats. Since fiona reads the extension of the supplied path when determining the driver to use, this same method works for both Geopackages and Geodatabases.

In [59]:
def list_layers(path):
    '''Returns a list of the layer names (str) in a GeoPackage or Geodatabase.'''
    return fiona.listlayers(path)

layer_names = list_layers(citibike_gpkg_path)

print(layer_names)

['bike_routes', 'street_centerline', 'vision_zero', 'census']


The available drivers for fiona are shown below. 'OpenFileGDB' is the appropriate driver for ESRI Geodatabases, but 'r' indicates that fiona can only read using this driver, not write.

I think there's a way around this to write to a Geodatabse with fiona, but here we'll just use arcpy.

In [55]:
fiona.supported_drivers

{'ARCGEN': 'r',
 'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'rw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'GPSTrackMaker': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

### Copying the features

Next, I defined a function to copy the features from a Geopackage to a Geodatabase.

In [46]:
def gpkg_to_gdb(gpkg_path, gdb_path, overwrite = True):
    # Create new GeoDatabase if none exists at the path
    if not arcpy.Exists(gdb_path):
        arcpy.management.CreateFileGDB(data_dir, gdb_name)
    
    layer_names = gpkg_layers(gpkg_path)
    
    for layer_name in layer_names:
        #Get paths of layers in GeoPackage and Geodatabase
        gpkg_layer_path = os.path.join(gpkg_path, f"main.{layer_name}")
        gdb_layer_path = os.path.join(gdb_path, layer_name)
        #Delete existing layer if overwrite=True
        if arcpy.Exists(gdb_layer_path) and overwrite:
            arcpy.Delete_management(gdb_layer_path)
        #Copy features from GeoPackage layer to Geodatabase layer
        arcpy.management.CopyFeatures(gpkg_layer_path, gdb_layer_path)

### Defining the CRS

It is also necessary to define the coordinate reference system for the feature classes in the new Geodatabase.

In [50]:
def gdb_define_proj(gdb_path, proj_code):
    '''Defines the projection of all the layers in an ESRI GeoDatabase'''
    # Get the names of the layers in the geodatabase 
    layer_names = gpkg_layers(gdb_path)

    # Iterate through the layers in the Geodatabase
    for layer_name in layer_names:
        gdb_layer_path = os.path.join(gdb_path, layer_name)
        print(gdb_layer_path)
        # Define the projection for the feature class
        arcpy.management.DefineProjection(gdb_layer_path, arcpy.SpatialReference(proj_code))

arcpy.env.workspace = data_dir
gpkg_to_gdb(citibike_gpkg_path, citibike_gdb_path, overwrite=True)
gdb_define_proj(citibike_gdb_path, 4326)

D:\bike_data\v3.gdb
D:\bike_data\v3.gdb\bike_routes
D:\bike_data\v3.gdb\street_centerline
D:\bike_data\v3.gdb\vision_zero
D:\bike_data\v3.gdb\census


Now, the Geodatabase can be added directly to an ArcGIS Pro project or used for additional arcpy scripting. 